# PyTorch Flowers Project

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms import ToTensor
import scipy


In [3]:
#flowertensor = torchvision.io.read_image("C:/Users/megan/Desktop/flowers-102/jpg/image_00001.jpg")
train_flowers = torchvision.datasets.Flowers102(root="C:/Users/megan/Desktop", split="train", download=True
                                                , transform=ToTensor()
                                                )
test_flowers = torchvision.datasets.Flowers102(root="C:/Users/megan/Desktop", split="test", download=True
                                               , transform=ToTensor()
                                              )


In [13]:
batch_size = 2

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data_utils import default_collate_fn_map

# Extend this function to handle batch of tensors
def collate_tensor_fn(batch, *, collate_fn_map):
    return torch.stack(batch, 0)
def custom_collate(batch):
    collate_map = {torch.Tensor: collate_tensor_fn}
    return collate(batch, collate_fn_map=collate_map)
# Extend `default_collate` by in-place modifying `default_collate_fn_map`
default_collate_fn_map.update({torch.Tensor: collate_tensor_fn})

# create data loader
train_dataloader = DataLoader(train_flowers, collate_fn=collate_fn, batch_size=batch_size)

#train_dataloader = pad_sequence(DataLoader(train_flowers, batch_size=batch_size))
test_dataloader = DataLoader(test_flowers, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X []: {X.shape}")
    print(f"Shape of y: {y.dtype}")
    break


ModuleNotFoundError: No module named 'torch.utils.data_utils'

In previous, tensor size is not equal, so it torch can't compare them, i think this is because the images aren't sized all the same and the number of dimensions of the 3rd parameter is inconsistent 

problem is in collate_fn 
https://pytorch.org/docs/stable/data.html#dataloader-collate-fn